In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
data = pd.concat([categorical,numerical],axis=1)

In [3]:
data

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,...,vehicle_class,vehicle_size,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,Washington,No,0.0,0.5,1.0,F,0.5,Married,Corporate Auto,L3,...,Two-Door Car,0.5,2763.519279,56274.0,69.0,32.0,5.0,0.0,1.0,384.811147
1,Arizona,No,0.5,0.5,0.0,F,0.5,Single,Personal Auto,L3,...,Four-Door Car,0.5,6979.535903,0.0,94.0,13.0,42.0,0.0,8.0,1131.464935
2,Nevada,No,1.0,0.5,1.0,F,0.5,Married,Personal Auto,L3,...,Two-Door Car,0.5,12887.431650,48767.0,108.0,18.0,38.0,0.0,2.0,566.472247
3,California,No,0.0,0.5,0.0,M,0.5,Married,Corporate Auto,L2,...,SUV,0.5,7645.861827,0.0,106.0,18.0,65.0,0.0,7.0,529.881344
4,Washington,No,0.0,0.5,1.0,M,0.0,Single,Personal Auto,L1,...,Four-Door Car,0.5,2813.692575,43836.0,73.0,12.0,44.0,0.0,1.0,138.130879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,California,No,0.0,0.5,1.0,M,1.0,Married,Personal Auto,L1,...,Four-Door Car,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9130,California,Yes,0.5,0.5,1.0,F,0.5,Divorced,Corporate Auto,L3,...,Four-Door Car,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9131,California,No,0.5,0.5,0.0,M,0.5,Single,Corporate Auto,L2,...,Four-Door Car,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9132,California,No,0.5,0.5,1.0,M,0.5,Married,Personal Auto,L2,...,Four-Door Car,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#X-Y Split, train-test split
X = data.drop(['total_claim_amount'],axis=1)
y = data['total_claim_amount']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train_num = X_train.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(object)

#cat/num X-test
X_test_num = X_test.select_dtypes(include = np.number)
X_test_cat = X_test.select_dtypes(object)

from sklearn.preprocessing import MinMaxScaler
#Scale X-train
transformer = MinMaxScaler().fit(X_train_num)
X_scaled_train = pd.DataFrame(transformer.transform(X_train_num),columns=X_train_num.columns)

#Scale X-test
X_scaled_test = pd.DataFrame(transformer.transform(X_test_num),columns=X_test_num.columns)
X_scaled_test

,coverage,education,employmentstatus,location_code,renew_offer_type,vehicle_size,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
0,0.0,0.0,0.5,0.0,0.333333,0.5,0.032205,0.418409,0.177215,0.800000,0.434343,0.0,0.000
1,0.0,0.0,0.5,0.5,1.000000,0.5,0.054785,0.232224,0.042194,0.200000,0.101010,0.0,0.375
2,0.5,0.5,0.5,0.5,0.333333,0.0,0.048458,0.228903,0.016878,0.257143,0.818182,0.0,0.250
3,0.5,0.5,0.5,1.0,0.000000,0.5,0.037170,0.741771,0.198312,0.400000,0.636364,0.0,0.000
4,0.0,0.0,1.0,0.5,0.000000,0.5,0.197235,0.000000,0.544304,0.028571,0.363636,0.0,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,0.0,0.5,0.5,0.5,0.333333,0.5,0.094175,0.306008,0.033755,0.628571,0.363636,0.0,0.125
1823,0.0,0.0,0.5,0.0,1.000000,1.0,0.083774,0.783089,0.130802,0.171429,0.353535,0.0,0.250
1824,0.0,0.5,0.5,0.0,0.000000,0.5,0.109032,0.651154,0.227848,0.828571,0.414141,0.8,0.250
1825,0.0,0.0,0.0,0.5,0.000000,0.0,0.121262,0.000000,0.303797,0.114286,0.585859,0.0,0.875


In [7]:
X_test_cat

,state,response,gender,marital_status,policy_type,policy,sales_channel,vehicle_class
708,California,No,M,Married,Personal Auto,L3,Agent,SUV
47,Oregon,No,F,Single,Personal Auto,L2,Agent,Two-Door Car
3995,Oregon,Yes,F,Married,Personal Auto,L3,Agent,Two-Door Car
1513,California,No,F,Married,Personal Auto,L3,Branch,Four-Door Car
3686,Washington,No,F,Divorced,Personal Auto,L2,Branch,Four-Door Car
...,...,...,...,...,...,...,...,...
4855,Oregon,No,F,Divorced,Personal Auto,L3,Agent,SUV
1880,Arizona,No,M,Married,Personal Auto,L2,Branch,Two-Door Car
8472,Oregon,No,F,Married,Corporate Auto,L3,Branch,SUV
5967,Washington,No,M,Single,Personal Auto,L2,Agent,Four-Door Car


In [8]:

# X_train_cat["coverage"] = X_train_cat["coverage"].map({"Basic" : 0, "Extended" :.5, "Premium" : 1})
# X_test_cat["coverage"] = X_test_cat["coverage"].map({"Basic" : 0, "Extended" :.5, "Premium" : 1})
# X_train_cat["employmentstatus"] = X_train_cat["employmentstatus"].map({"Medical Leave" : 0, "Disabled" : 0, "Unemployed" : 0, "Retired" : .5, "Employed":1})
# X_test_cat["employmentstatus"] = X_test_cat["employmentstatus"].map({"Medical Leave" : 0, "Disabled" : 0, "Unemployed" : 0, "Retired" : .5, "Employed":1})
# X_train_cat["location_code"] =X_train_cat["location_code"].map({"Rural" : 0, "Suburban" :.5, "Urban" : 1,})
# X_test_cat["location_code"] =X_test_cat["location_code"].map({"Rural" : 0, "Suburban" :.5, "Urban" : 1,})
# X_train_cat["vehicle_size"] =X_train_cat["vehicle_size"].map({"Small" : 0, "Medsize" :.5, "Large" : 1,})
# X_test_cat["vehicle_size"] =X_test_cat["vehicle_size"].map({"Small" : 0, "Medsize" :.5, "Large" : 1,})
# X_train_cat["education"] = X_train_cat["education"].map({"High School or Below" : 0, "College" :.33, "Bachelor" : .66,"Post-Uni" : 1})
# X_test_cat["education"] = X_test_cat["education"].map({"High School or Below" : 0, "College" :.33, "Bachelor" : .66,"Post-Uni" : 1})


X_train_1_hot_col = X_train[['state', 'policy','response','gender','marital_status','policy_type','renew_offer_type','sales_channel','vehicle_class']]
X_test_1_hot_col = X_test[['state', 'policy', 'response','gender','marital_status','policy_type','renew_offer_type','sales_channel','vehicle_class']]

encoder = OneHotEncoder(drop='first', sparse_output=False).fit(X_train_1_hot_col)

# Get the new column names
cols = encoder.get_feature_names_out(input_features=X_train_1_hot_col.columns)

# Transform and create new DataFrames
X_train_col_encoded = pd.DataFrame(encoder.transform(X_train_1_hot_col), columns=cols)
X_test_col_encoded = pd.DataFrame(encoder.transform(X_test_1_hot_col), columns=cols)

# Concatenate the new columns to the original DataFrames
encoded_train = pd.concat([X_train.drop(columns=X_train_1_hot_col.columns), X_train_col_encoded], axis=1)
encoded_test = pd.concat([X_test.drop(columns=X_test_1_hot_col.columns), X_test_col_encoded], axis=1)


In [9]:
X_train

,state,response,coverage,education,employmentstatus,gender,location_code,marital_status,policy_type,policy,...,sales_channel,vehicle_class,vehicle_size,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
5123,Washington,No,0.0,0.5,1.0,M,1.0,Married,Personal Auto,L3,...,Agent,Two-Door Car,0.5,2552.038766,75740.0,64.0,13.0,11.0,0.0,1.0
7738,Oregon,No,0.5,0.5,0.0,F,0.5,Single,Special Auto,L2,...,Branch,Four-Door Car,0.5,10793.757040,24195.0,90.0,32.0,70.0,0.0,2.0
214,Arizona,No,0.0,0.5,1.0,M,0.5,Divorced,Personal Auto,L1,...,Agent,Four-Door Car,0.0,2912.892035,81097.0,74.0,7.0,67.0,0.0,1.0
8580,California,No,0.0,0.5,1.0,F,1.0,Married,Corporate Auto,L3,...,Web,Four-Door Car,0.5,10408.968650,20228.0,101.0,3.0,43.0,1.0,2.0
7857,Washington,No,0.0,0.5,1.0,F,0.5,Single,Personal Auto,L3,...,Web,Four-Door Car,0.5,5875.212734,46417.0,74.0,16.0,14.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,California,No,0.0,0.5,1.0,F,0.5,Divorced,Personal Auto,L3,...,Agent,Four-Door Car,0.0,2843.557702,67952.0,71.0,16.0,0.0,0.0,1.0
5191,California,Yes,0.0,0.5,2.0,M,0.5,Married,Personal Auto,L3,...,Branch,Four-Door Car,0.5,4921.325084,43640.0,64.0,10.0,99.0,2.0,3.0
5390,California,No,0.5,0.0,0.0,F,0.5,Single,Personal Auto,L1,...,Call Center,SUV,0.5,8257.666512,23815.0,73.0,4.0,47.0,0.0,2.0
860,Arizona,Yes,0.5,0.5,0.0,F,0.5,Divorced,Personal Auto,L3,...,Call Center,SUV,0.5,5609.082494,44705.0,71.0,21.0,63.0,0.0,6.0


In [10]:
X_test = X_test.reset_index()

In [11]:
X_train = X_train.reset_index()

In [20]:
train_scaled = pd.concat([encoded_train,X_scaled_train],axis=1)
test_scaled = pd.concat([encoded_test,X_scaled_test],axis=1)
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

DT = DecisionTreeRegressor()
LR = LinearRegression()
RF = RandomForestRegressor()

model_pipeline = [DT, LR, RF]
model_names = ['Regression Tree', 'Linear Regression','Random Forest']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, train_scaled, y_train, cv=10))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

ValueError: Found input variables with inconsistent numbers of samples: [8789, 7307]

In [21]:
val_scores = {}
i=0
for model in model_pipeline:
    model.fit(train_scaled, y_train)
    val_scores[model_names[i]] = model.score(test_scaled,y_test)
    i = i+1
print(val_scores)

ValueError: Input X contains NaN.
DecisionTreeRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# I understand that there is a problem with the nan values and I should check the cleaning process again.